In [ ]:
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
me.init_vprinting()

t = me.dynamicsymbols._t
m, mw, Ixx, Iyy, Izz, h, g, a, b = sm.symbols('m, mw, I_{xx}, I_{yy}, I_{zz}, h, g, a, b')
theta, r, psi, y, x, z, xB, yB, delta = me.dynamicsymbols('theta, r, psi, y, x, z, xB, yB, delta')
q1, q2, q3, q4, u1, u2, u3, u4 = me.dynamicsymbols('q1, q2, q3, q4, u1, u2, u3, u4')
mu = me.dynamicsymbols('mu') 
N, B, A, F = sm.symbols('N, B, A, F', cls=me.ReferenceFrame)
#Fyf, Fyb, alpha_f, alpha_b = sm.symbols('Fyf, Fyb, alpha_f, alpha_b')
Bt, Ct, Dt, Et = sm.symbols('Bt, Ct, Dt, Et')
Bb, Cb, Db, Eb = sm.symbols('Bb, Cb, Db, Eb')

# Frames
A.orient_axis(N, r, N.z)
B.orient_axis(A, theta, A.x)
F.orient_axis(A, delta, A.z)

#q = sm.Matrix([theta, r, xB, yB, delta])# this should change since x,y,z,xB, yB can all relate to angles?
q1 = theta
q2 = r
q3 = xB
q4 = yB

# these must be like this not theta = q1 otherwise the substitution gets weird
q = sm.Matrix([theta, r, xB, yB])

#q = sm.Matrix([theta, r, xB, yB])
qd = q.diff(t)
qdd = qd.diff(t)

u = sm.Matrix([u1, u2, u3, u4])# all generalized speeds
ud = u.diff(t)
ur = sm.Matrix([u2, u3]) # dependent 
us = sm.Matrix([u4]) # independent 
urd = ur.diff(t)
usd = us.diff(t)
p = sm.Matrix([Ixx, Iyy, Izz, g, h, m, mw, t, a, b, Ct, Cb, Bt, Ct, Dt, Et])# constants in my equation

steer = sm.Matrix([delta])

#setting up my zeors
qd_zero = {qdi: 0 for qdi in qd} 
ur_zero = {uri: 0 for uri in ur} 
u_zero = {ui: 0 for ui in u}
urd_zero = {urdi: 0 for urdi in urd} 
usd_zero = {usdi: 0 for usdi in usd}
ud_zero = {udi: 0 for udi in ud}

######### Kinematic equations formulation ############
fk = sm.Matrix([
    u1 - q1.diff(t),
    u2 - q2.diff(t),
    u3 - q3.diff(t),
    u4 - q4.diff(t)])

### Solvinig the Kin eq for qd to change them to terms with u's #####
Mk = fk.jacobian(qd)
gk = fk.xreplace(qd_zero)
qd_sol = -Mk.LUsolve(gk)
qd_repl = dict(zip(qd, qd_sol))

### All of my points being set up ####
Base = me.Point('Base')
O = me.Point('O')
COM = me.Point('COM')
COMp = me.Point('COMp')
Wf = me.Point('Wf')
Wb = me.Point('Wb')

# telling python where my points are realtive to each other #
Base.set_pos(O, q3*N.x + q4*N.y+0*N.z)#changes with time so they can be q's?
COM.set_pos(Base, -h*B.z)#doesnt change with time
COMp.set_pos(COM, h*sm.cos(q1)*-N.z)#h.dot(-N.z)*N.z
Wf.set_pos(Base, a*B.x+0*N.z)#a is fixed dist from COM to wheel gnd cont pnt
Wb.set_pos(Base, -b*B.x+0*N.z)#b is a fixed dist from COM to wheel gnd cont pnt

# getting velocities of COM #
O.set_vel(N, 0)
Wf.set_vel(F, 0)
N_v_base = Base.pos_from(O).diff(t, N)#need this for 2pt thing
Base.set_vel(N, N_v_base)
N_v_Wb = Wb.v2pt_theory(Base, N, A).xreplace(qd_repl)
N_v_Wf = Wf.v2pt_theory(Base, N, A).xreplace(qd_repl)
N_v_COM = COM.v2pt_theory(Base, N, B) #this is U


# angular velocity of my body #
N_w_B = B.ang_vel_in(N)

#stuff for 1/2*I*w^2 part of KE
I_B = me.inertia(B, Ixx, Iyy, Izz) #eventually add all 9 terms


N_v_Wf, N_v_Wb

# vel and ang vel with qd replaced to be u
#N_v_COM = COM.v2pt_theory(Base, N, B).xreplace(qd_repl)
#N_w_B = B.ang_vel_in(N).xreplace(qd_repl)


In [ ]:
nh = sm.Matrix([N_v_Wf.dot(F.y),
                N_v_Wb.dot(A.y)])# change the second to an intermediate frame
Mn = nh.jacobian(ur)
gn = nh.xreplace(ur_zero)\
#ur_sol = sm.trigsimp(-Mn.LUsolve(gn))
#ur_repl = dict(zip(ur, ur_sol))
# finding the nonholo constraints as longitude and lateral slip
fnd = nh.diff(t).xreplace(qd_repl)# did qd_repl not work?
Mnd = fnd.jacobian(ud)
gnd = me.msubs(fnd, {u1.diff(t):0,u2.diff(t):0, u3.diff(t):0, u4.diff(t):0, delta.diff(t):0 })

# the non holo constraint (vel terms) and trying to get Mn, gn

In [ ]:
# def calculate_Q(steer_vals, N_v_Wf, N_v_Wb, Ct, Dt, Bt, Et):
#   """
#   Calculates the Q matrix in the vehicle dynamics equations,
#   which relates lateral tire forces to generalized coordinates.
#   """

#   # Calculate slip angles at the front and rear tires
#   alpha_f = delta - sm.atan2(N_v_Wf.dot(F.y), N_v_Wf.dot(F.x))
#   alpha_b = -sm.atan2(N_v_Wb.dot(A.y), N_v_Wb.dot(A.x))

#   # Use Magic Formula to calculate lateral forces
#   Fyf = Dt * sm.sin(Ct * sm.atan(Bt * alpha_f - Et * (Bt * alpha_f - sm.atan(Bt * alpha_f))))
#   Fyb = Dt * sm.sin(Ct * sm.atan(Bt * alpha_b - Et * (Bt * alpha_b - sm.atan(Bt * alpha_b))))

#   Q1 = Fyf * Wf.pos_from(O).diff(q1, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q1, N).dot(A.y)
#   Q2 = Fyf * Wf.pos_from(O).diff(q2, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q2, N).dot(A.y)
#   Q3 = Fyf * Wf.pos_from(O).diff(q3, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q3, N).dot(A.y)
#   Q4 = Fyf * Wf.pos_from(O).diff(q4, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q4, N).dot(A.y)

#   Q = sm.Matrix([ Q1, Q2, Q3, Q4])

#   return Q

### add in steer
# # my constraint saying there will be no movement in Y #
# nh_new = sm.Matrix([N_v_Wf.dot(F.y) - Fyf,
#                     N_v_Wb.dot(A.y) - Fyb])# change the second to an intermediate frame

# Mn_new = nh_new.jacobian(ur)
# gn_new = nh_new.xreplace(ur_zero)

# # finding the nonholo constraints as longitude and lateral slip
# fnd_new = nh_new.diff(t).xreplace(qd_repl)# did qd_repl not work?
# Mnd_new = fnd_new.jacobian(ud)
# gnd_new = me.msubs(fnd_new, {u1.diff(t):0,u2.diff(t):0, u3.diff(t):0, u4.diff(t):0, delta.diff(t):0 })


In [ ]:
########## nonholo const of wheel ##########

# Static vertical load on front and rear axles
F_zf = (m * g * b) / (a + b)
F_zb = (m * g * a) / (a + b)

# Maximum lateral forces due to friction

Fyf_f = mu * F_zf
Fyb_f = mu * F_zb

# Limit slip angles for realistic values
 # Max slip angle (45 degrees)
alpha_f = delta - sm.atan2(N_v_Wf.dot(F.y), N_v_Wf.dot(F.x))
alpha_b = -sm.atan2(N_v_Wb.dot(B.y), N_v_Wb.dot(B.x))

# Apply the Magic Formula with friction limits
Fyf = Dt * sm.sin(Ct * sm.atan(Bt * alpha_f - Et * (Bt * alpha_f - sm.atan(Bt * alpha_f))))
Fyb = Dt * sm.sin(Ct * sm.atan(Bt * alpha_b - Et * (Bt * alpha_b - sm.atan(Bt * alpha_b))))


# # Calculate slip angles at the front and rear tires
# alpha_f = (delta - sm.atan2(u4 + a* u2, u3))
# alpha_b = -sm.atan2(u4 - b*u2, u3)

# # simple linear lateral force
# #Fyf = Cf * alpha_f
# #Fyb = Cb * alpha_b


# # Use Magic Formula to calculate lateral forces
# Fyf = Dt * sm.sin(Ct * sm.atan(Bt * alpha_f - Et * (Bt * alpha_f - sm.atan(Bt * alpha_f))))
# Fyb = Dt * sm.sin(Ct * sm.atan(Bt * alpha_b - Et * (Bt * alpha_b - sm.atan(Bt * alpha_b))))

Q1 = Fyf * Wf.pos_from(O).diff(q1, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q1, N).dot(A.y) + Fyf_f * Wf.pos_from(O).diff(q1, N).dot(F.y) + Fyb_f * Wb.pos_from(O).diff(q1, N).dot(A.y)
Q2 = Fyf * Wf.pos_from(O).diff(q2, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q2, N).dot(A.y) + Fyf_f * Wf.pos_from(O).diff(q2, N).dot(F.y) + Fyb_f * Wb.pos_from(O).diff(q2, N).dot(A.y)
Q3 = Fyf * Wf.pos_from(O).diff(q3, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q3, N).dot(A.y) + Fyf_f * Wf.pos_from(O).diff(q3, N).dot(F.y) + Fyb_f * Wb.pos_from(O).diff(q3, N).dot(A.y)
Q4 = Fyf * Wf.pos_from(O).diff(q4, N).dot(F.y) + Fyb * Wb.pos_from(O).diff(q4, N).dot(A.y) + Fyf_f * Wf.pos_from(O).diff(q4, N).dot(F.y) + Fyb_f * Wb.pos_from(O).diff(q4, N).dot(A.y)

# Q1 = Fyf * (u4 + a * u2) + Fyb * (u4 - b * u2)
# Q2 = Fyf * (a * u2) + Fyb * (-b * u2)
# Q3 = Fyf * u3 + Fyb * u3 
# Q4 = Fyf * (u4 + a * u2) + Fyb * (u4 - b * u2)

Q = sm.Matrix([ Q1, Q2, Q3, Q4])


### add in steer
# my constraint saying there will be no movement in Y #
nh_new = sm.Matrix([N_v_Wf.dot(F.y) - (-Fyf + Fyf_f),
                    N_v_Wb.dot(A.y) - (-Fyb + Fyb_f)])# change the second to an intermediate frame

Mn_new = nh_new.jacobian(ur)
gn_new = nh_new.xreplace(ur_zero)

# finding the nonholo constraints as longitude and lateral slip
fnd_new = nh_new.diff(t).xreplace(qd_repl)# did qd_repl not work?
Mnd_new = fnd_new.jacobian(ud)
gnd_new = me.msubs(fnd_new, {u1.diff(t):0,u2.diff(t):0, u3.diff(t):0, u4.diff(t):0, delta.diff(t):0, mu.diff(t):0 })

me.find_dynamicsymbols(gnd_new)
gnd_new


 # Finding KE, PE and Lagrange

In [ ]:
# KE EQ

#vel_lat = N_v_COM*(Wf.pos_from(COM).dot(F.x) + Wb.pos_from(COM).dot(B.x))/(a+b)# I think this is lateral velocity
#vel = N_v_COM +vel_lat

#KE = (m/2*(N_v_COM.dot(N_v_COM))) + (N_w_B.dot(I_B.dot(N_w_B))/2)
#KE.simplify()

KE_trans = m/2*(N_v_COM.dot(N_v_COM))
KE_rot = N_w_B.dot(I_B.dot(N_w_B))/2


#N_v_Wf_s = Fyf*(F.y) + N_v_Wf
#N_v_Wb_s = Fyf*(A.y) + N_v_Wb

KE_front = (mw / 2) * (N_v_Wf.dot(N_v_Wf))
KE_rear = (mw / 2) * (N_v_Wf.dot(N_v_Wf)) 

KE = KE_trans + KE_rot + KE_front + KE_rear
KE=sm.trigsimp(KE)# the I part rdot the book has r and the mass part seems wrong 


#Potential Energy
#PE=mgh
#heigth = COM.pos_from(O).express(N).dot(N.z)# <<me figuring out why it didnt work before
PE = m*g*-COM.pos_from(O).express(N).dot(N.z)

#lagrange part
L = sm.Matrix([KE - PE])
sm.trigsimp(L)

# Update the equations of motion
qdd_zerod = {qddr: 0 for qddr in qdd}
fd = -(L.jacobian(qd).diff(t) - L.jacobian(q)).transpose()

# Compute the mass matrix and generalized forces for the updated system
Md = fd.jacobian(qdd)
gd = sm.trigsimp(fd.xreplace(qdd_zerod)).xreplace(qd_repl)

# Update the lagrange system with generalized forces
Md_lagrange = sm.Matrix([[Md, Mnd.transpose()], [Mnd, sm.zeros(2, 2)]])
gd_lagrange = sm.Matrix([gd, gnd])
### I need the lagrange multipliers now
#Md.free_symbols|gd.free_symbols

Md_lagrange, gd#why do I not have mass terms in xdd and ydd


In [ ]:

KE_trans = m/2*(N_v_COM.dot(N_v_COM))
KE_rot = N_w_B.dot(I_B.dot(N_w_B))/2


#N_v_Wf_s = Fyf*(F.y) + N_v_Wf
#N_v_Wb_s = Fyf*(A.y) + N_v_Wb

#KE_front = (mw / 2) * (N_v_Wf_s.dot(N_v_Wf_s))
#KE_rear = (mw / 2) * (N_v_Wf_s.dot(N_v_Wf_s)) 

KE_new= KE_trans + KE_rot# + KE_front + KE_rear
KE=sm.trigsimp(KE)# the I part rdot the book has r and the mass part seems wrong 


#Potential Energy
#PE=mgh
#heigth = COM.pos_from(O).express(N).dot(N.z)# <<me figuring out why it didnt work before
PE_new = m*g*-COM.pos_from(O).express(N).dot(N.z)

#lagrange part
L_new = sm.Matrix([KE_new - PE_new])
sm.trigsimp(L)

# Update the equations of motion
qdd_zerod = {qddr: 0 for qddr in qdd}
fd_new = -(L_new.jacobian(qd).diff(t) - L_new.jacobian(q) ).transpose() 

#Compute the mass matrix and generalized forces for the updated system
Md_new = fd_new.jacobian(qdd)
gd_new = fd_new.xreplace(qdd_zerod).xreplace(qd_repl) + Q


# Update the lagrange system with generalized forces
Md_lagrange_new = sm.Matrix([[Md_new, Mnd_new.transpose()], [Mnd_new, sm.zeros(2, 2)]])
gd_lagrange_new = sm.Matrix([gd_new, gnd_new])#.xreplace({mu.diff(t):mu})
## I need the lagrange multipliers now
Md_lagrange_new.free_symbols|Md_lagrange.free_symbols

me.find_dynamicsymbols(gd_lagrange_new)
#gd_lagrange_new

# simulation setup:
 I need the MD,gd, Mk, gk and I think the Mnd, gnd

In [ ]:

import numpy as np

#inputs
x0 = np.array([
    np.deg2rad(180), #q1 theta, lean
    np.deg2rad(8), #q2 r ang_vel
    0.1, #q3 x
    0.1, #q4 y
    np.deg2rad(4), #u1 theta
    np.deg2rad(10), #u2 r ang_acc
    2, #u3 x
    0.1 #u4 y 
])

p_vals = np.array([
    0.1, #Ixx
    0.1, #Iyy
    0.1, #Izz
    9.8, #g
    5, #h
    15, #m
    1, #mw
    0.1, #t
    3, #a
    3, #b
    1.3,#Cf
    1.3, #Cb
    10, # Bt
    1.3, #Ct
    1, #Dt
    0.97 #Et
    
   
])

steer_vals = np.array([
    np.deg2rad(45) #delta
    
])


###### solving for my dependent u2 and u3
#eval_Mngn = sm.lambdify((q, u, p, steer), (Mn, gn))
#u_vals = -np.linalg.solve(*eval_Mngn(q_vals, u_vals, p_vals, steer_vals))# solve for dependent u's, u2, u3


###### solving for my states
eval_lagrange = sm.lambdify((q, u, p, steer), [Mk, gk, Md_lagrange, gd_lagrange], cse=True)

def eval_rhs_lagrange(t, x, p, steer_vals):
    q = x[0:4]
    u = x[4:8]
    roll_angle = q[0]
    steer_angle = [-2 * roll_angle]

    Mk_vals, gk_vals, Md_lagrange, gd_lagrange = eval_lagrange(q, u, p, steer_angle)
    
    qd = np.linalg.solve(-Mk_vals, np.squeeze(gk_vals))
    ud_and_lam = np.linalg.solve(-Md_lagrange, np.squeeze(gd_lagrange))
    
    ud = ud_and_lam[0:4]
    lam = ud_and_lam[4:6]
    
    return np.hstack((qd, ud)), lam

# # Now solve the system with forces applied to Wf and Wb
# Fyf_val = 10.0  # Example lateral force at Wf
# Fyb_val = 10.0  # Example lateral force at Wb

t0, tf = 0.0, 100.0
fps = 30
ts = np.linspace(t0, tf, num=int((tf-t0)*fps))

sol_lagrange = solve_ivp(
    lambda t, x, p: eval_rhs_lagrange(t, x, p, steer_vals)[0],
    (t0, tf), x0, args=(p_vals,), t_eval=ts
)



In [ ]:
# ###### solving for my states
# eval_lagrange_new = sm.lambdify((q, u, p, steer), [Mk, gk, Md_lagrange_new, gd_lagrange_new], cse=True)

# def eval_rhs_lagrange_new(t,  x, p, others):
#     q = x[0:4]
#     u = x[4:8]
#     roll_angle = q[0]
#     steer_angle = [-2 * roll_angle]

#     steer_angle = [-2 * roll_angle]
#     delta = others[0]  # Steering angle
#     mu = others[1]     # Friction coefficient



#     Mk_vals, gk_vals, Md_lagrange_new, gd_lagrange_new = eval_lagrange_new(q, u, p, steer_angle)
    
#     qd = np.linalg.solve(-Mk_vals, np.squeeze(gk_vals))
#     ud_and_lam_new = np.linalg.solve(-Md_lagrange_new, np.squeeze(gd_lagrange_new))
    
#     ud_new = ud_and_lam_new[0:4]
#     lam_new = ud_and_lam_new[4:6]
    
#     return np.hstack((qd, ud_new)), lam_new

# t0, tf = 0.0, 100.0
# fps = 30
# ts = np.linspace(t0, tf, num=int((tf-t0)*fps))

# sol_lagrange_new = solve_ivp(
#     lambda t, x, p: eval_rhs_lagrange_new(t, x, p, others)[0],
#     (t0, tf), x0, args=(p_vals,), t_eval=ts
# )

In [ ]:
# Convert 45 degrees to radians

eval_lagrange_new = sm.lambdify((q, u, p, steer, mu), [Mk, gk, Md_lagrange_new, gd_lagrange_new])

# Modify the RHS function to handle the time window for slip angles
def eval_rhs_lagrange_new(t, x, p, steer_vals):
    q = x[0:4]
    u = x[4:8]
    roll_angle = q[0]
    steer_angle = [-2 * roll_angle]

    # # Conditional logic to set mu based on the time t
    mu = 0.04
    # if 5 <= t <= 10:
    #      mu = -0.4  # No friction between t=5 and t=10
  

    # Update the state based on the modified or original slip angles
    Mk_vals, gk_vals, Md_lagrange_new, gd_lagrange_new = eval_lagrange_new(q, u, p, steer_angle, mu)
    
    qd = np.linalg.solve(-Mk_vals, np.squeeze(gk_vals))
    ud_and_lam_new = np.linalg.solve(-Md_lagrange_new, np.squeeze(gd_lagrange_new))
    
    ud_new = ud_and_lam_new[0:4]
    lam_new = ud_and_lam_new[4:6]
    
    return np.hstack((qd, ud_new)), lam_new
t0, tf = 0.0, 100.0
fps = 30
ts = np.linspace(t0, tf, num=int((tf-t0)*fps))

# Solve the system using the modified RHS function
sol_lagrange_new = solve_ivp(
    lambda t, x, p: eval_rhs_lagrange_new(t, x, p, steer_vals)[0],
    (t0, tf), x0, args=(p_vals,), t_eval=ts
)
### check the EOM holo motion page to see how he did risidual forces ###

In [ ]:
#eval_eom = sm.lambdify((q, u, p, steer), [Mk, gk, Md, gd])
#Mk_vals, gk_vals, Md_vals, gd_vals = eval_eom(x0[0:4],x0[4:8], p_vals, steer_vals)

#Mk_vals, gk_vals, Md_vals, gd_vals

#qd_vals = np.linalg.solve(-Mk_vals, np.squeeze(gk_vals))
#qd_vals

#ud_vals = np.linalg.solve(-Md_vals, np.squeeze(gd_vals))
#ud_vals


below is me checking every step of the thing above to see what I got, it can be removed

In [ ]:
#Mk_vals, gk_vals, Md_vals, gd_vals = eval_lagrange(q_vals, u_vals, p_vals, steer_vals)
#ud_and_lam = np.linalg.solve(-Md_vals, np.squeeze(gd_vals))
#ud_and_lam
#ud = ud_and_lam[0:4]
#lam = ud_and_lam[4:6]
#ud, lam

#qd = np.linalg.solve(-Mk_vals, np.squeeze(gk_vals))
#qd

#np.hstack((qd, ud))

In [ ]:
def euler_integrate(rhs_func, tspan, x0, p_vals, steer_vals, delt=0.03):
    """Returns state trajectory and corresponding values of time resulting
    from integrating the ordinary differential equations with Euler's
    Method.

    Parameters
    ==========
    rhs_func : function
       Python function that evaluates the derivative of the state and takes
       this form ``dxdt = f(t, x, p)``.
    tspan : 2-tuple of floats
       The initial time and final time values: (t0, tf).
    x0_vals : array_like, shape(2*n,)
       Values of the state x at t0.
    p_vals : array_like, shape(o,)
       Values of constant parameters.
    delt : float
       Integration time step in seconds/step.

    Returns
    =======
    ts : ndarray(m, )
       Monotonically equally spaced increasing values of time spaced apart
       by ``delt``.
    xs : ndarray(m, 2*n)
       State values at each time in ts.

    """
    # generate monotonically increasing values of time.
    duration = tspan[1] - tspan[0]
    num_samples = round(duration/delt) + 1
    ts = np.arange(tspan[0], tspan[0] + delt*num_samples, delt)

    # create an empty array to hold the state values.
    x = np.empty((len(ts), len(x0_vals)))

    # set the initial conditions to the first element.
    x[0, :] = x0_vals

    # use a for loop to sequentially calculate each new x.
    for i, ti in enumerate(ts[:-1]):
        x[i + 1, :] = x[i, :] + delt*rhs_func(ti, x[i, :], p_vals, steer_vals)

    return ts, x

In [ ]:
coordinates = O.pos_from(O).to_matrix(N)
for point in [Wf, Wb, Base, COM]:
    coordinates = coordinates.row_join(point.pos_from(O).to_matrix(N))

eval_point_coords = sm.lambdify((q, p), coordinates)
eval_point_coords(x0[0:4], p_vals)



########## the ones with slip ###########

coordinates_new = O.pos_from(O).to_matrix(N)
for point in [Wf, Wb, Base, COM]:
    coordinates_new = coordinates_new.row_join(point.pos_from(O).to_matrix(N))

eval_point_coords_new = sm.lambdify((q, p), coordinates_new)
eval_point_coords_new(x0[0:4], p_vals)


In [ ]:

# # initial configuration of the points
# x, y, z = eval_point_coords(x0[:4], p_vals)

# # create a figure
# fig = plt.figure()
# fig.set_size_inches((10.0, 10.0))

# # setup the subplots
# ax_3d = fig.add_subplot(1, 1, 1, projection='3d')

# # 3d view
# lines_3d, = ax_3d.plot(x, y, z)
# ax_3d.set_xlim((-100.0, 100.0))
# ax_3d.set_ylim((-100.0, 100.0))
# ax_3d.set_zlim((-10.0, 10.0))
# ax_3d.set_xlabel('x')
# ax_3d.set_ylabel('y')
# ax_3d.set_zlabel('z')

# fig.tight_layout()

# plt.close()

# # Animation 
# from matplotlib.animation import FuncAnimation

# def animate(i):
#     x, y, z = eval_point_coords(sol_lagrange.y.T[i, :4], p_vals)
#     lines_3d.set_data_3d(x, y, z)

# ani = FuncAnimation(fig, animate, len(sol_lagrange.t))

# from IPython.display import HTML
# import matplotlib
# matplotlib.rcParams['animation.embed_limit'] = 400.0
# HTML(ani.to_jshtml(fps=fps))

In [ ]:
# #initial configuration of the points
# x, y, z = eval_point_coords_new(x0[:4], p_vals)

# # create a figure
# fig = plt.figure()
# fig.set_size_inches((10.0, 10.0))

# # setup the subplots
# ax_3d = fig.add_subplot(1, 1, 1, projection='3d')

# # 3d view
# lines_3d, = ax_3d.plot(x, y, z)
# ax_3d.set_xlim((-100.0, 100.0))
# ax_3d.set_ylim((-100.0, 100.0))
# ax_3d.set_zlim((-10.0, 10.0))
# ax_3d.set_xlabel('x')
# ax_3d.set_ylabel('y')
# ax_3d.set_zlabel('z')

# fig.tight_layout()

# plt.close()

# # Animation 
# from matplotlib.animation import FuncAnimation

# def animate(i):
#     x, y, z = eval_point_coords_new(sol_lagrange_new.y.T[i, :4], p_vals)
#     lines_3d.set_data_3d(x, y, z)

# ani = FuncAnimation(fig, animate, len(sol_lagrange_new.t))

# from IPython.display import HTML
# import matplotlib
# matplotlib.rcParams['animation.embed_limit'] = 400.0
# HTML(ani.to_jshtml(fps=fps))

In [ ]:

# def extract_positions(solution, a, b):
#     """Extract the positions of the COM, Wf, and Wb from the solution, relative to the origin O."""
    
#     theta_vals = solution.y[0]  # Roll angle (θ)
#     r_vals = solution.y[1]      # Steering angle (r)
#     xO_vals = solution.y[2]     # x position of origin O
#     yO_vals = solution.y[3]     # y position of origin O

#     # Calculate the positions of COM
#     xB_vals = xO_vals + a * np.cos(theta_vals)  # x-position of COM
#     yB_vals = yO_vals + a * np.sin(theta_vals)  # y-position of COM

#     # Calculate the positions of Wf and Wb
#     Wf_x = xO_vals + (a + b) * np.cos(theta_vals + r_vals)  # x-position of Wf
#     Wf_y = yO_vals + (a + b) * np.sin(theta_vals + r_vals)  # y-position of Wf
    
#     Wb_x = xO_vals - b * np.cos(theta_vals)  # x-position of Wb
#     Wb_y = yO_vals - b * np.sin(theta_vals)  # y-position of Wb
    
#     return xO_vals, yO_vals, Wf_x, Wf_y, Wb_x, Wb_y


# # Extracting positions for both scenarios
# xB_no_slip, yB_no_slip, Wf_x_no_slip, Wf_y_no_slip, Wb_x_no_slip, Wb_y_no_slip = extract_positions(sol_lagrange, 3, 3)
# xB_with_slip, yB_with_slip, Wf_x_with_slip, Wf_y_with_slip, Wb_x_with_slip, Wb_y_with_slip = extract_positions(sol_lagrange_new, 3, 3)

# # Plotting the results
# plt.figure(figsize=(10, 8))

# # Plot for COM
# plt.plot(xB_no_slip, yB_no_slip, 'b-', label="COM Path (No Slip)")
# plt.plot(xB_with_slip, yB_with_slip, 'r--', label="COM Path (With Slip)")

# # Plot for Wf
# plt.plot(Wf_x_no_slip, Wf_y_no_slip, 'g-', label="Front Wheel Path (No Slip)")
# plt.plot(Wf_x_with_slip, Wf_y_with_slip, 'y--', label="Front Wheel Path (With Slip)")

# # Plot for Wb
# plt.plot(Wb_x_no_slip, Wb_y_no_slip, 'c-', label="Rear Wheel Path (No Slip)")
# plt.plot(Wb_x_with_slip, Wb_y_with_slip, 'm--', label="Rear Wheel Path (With Slip)")

# # Add legend and labels
# plt.xlabel("X Position")
# plt.ylabel("Y Position")
# plt.title("Comparison of Paths With and Without Slip")
# plt.legend()
# plt.grid(True)
# plt.axis('equal')
# plt.show()

In [ ]:
# from matplotlib.animation import FuncAnimation

# # Create a new figure for 2D X-Y plane plot
# fig_2d, ax_2d = plt.subplots()
# fig_2d.set_size_inches((10.0, 10.0))

# # Setup 2D plot limits
# ax_2d.set_xlim((-50.0, 50.0))
# ax_2d.set_ylim((-50.0, 50.0))
# ax_2d.set_xlabel('X')
# ax_2d.set_ylabel('Y')

# # Initialize scatter plots for Wf, Wb, and COM in 2D for both solutions (current and "No Slip")
# point_Wf_2d, = ax_2d.plot([], [], 'bo', label='Front Wheel (No Slip)')  # blue dot for Wf (slip)
# point_Wb_2d, = ax_2d.plot([], [], 'co', label='Rear Wheel (No Slip)')  # cyan dot for Wb (slip)
# point_COM_2d, = ax_2d.plot([], [], 'mo', label='Center of Mass (No Slip)')  # magenta dot for COM (slip)

# # For "No Slip" points (sol_lagrange_new)
# point_Wf_2d_s, = ax_2d.plot([], [], 'bx', label='Front Wheel (Slip)')  # blue cross for Wf (no slip)
# point_Wb_2d_s, = ax_2d.plot([], [], 'cx', label='Rear Wheel (Slip)')  # cyan cross for Wb (no slip)
# point_COM_2d_s, = ax_2d.plot([], [], 'mx', label='Center of Mass (Slip)')  # magenta cross for COM (no slip)

# # Add a legend
# ax_2d.legend()

# # Function to extract X, Y coordinates and update 2D plot for each frame
# def animate_2d(i):
#     # Current solution (with slip)
#     x, y, z = eval_point_coords(sol_lagrange.y.T[i, :4], p_vals)
#     x_Wf, y_Wf = x[1], y[1]  # Front wheel (slip)
#     x_Wb, y_Wb = x[2], y[2]  # Rear wheel (slip)
#     x_COM, y_COM = x[3], y[3]  # Center of Mass (slip)

#     # "No Slip" solution
#     x_ns, y_ns, z_ns = eval_point_coords(sol_lagrange_new.y.T[i, :4], p_vals)
#     x_Wf_ns, y_Wf_ns = x_ns[1], y_ns[1]  # Front wheel (no slip)
#     x_Wb_ns, y_Wb_ns = x_ns[2], y_ns[2]  # Rear wheel (no slip)
#     x_COM_ns, y_COM_ns = x_ns[3], y_ns[3]  # Center of Mass (no slip)

#     # Update the 2D scatter plot data for "With Slip" solution
#     point_Wf_2d.set_data(x_Wf, y_Wf)
#     point_Wb_2d.set_data(x_Wb, y_Wb)
#     point_COM_2d.set_data(x_COM, y_COM)

#     # Update the 2D scatter plot data for "No Slip" solution
#     point_Wf_2d_s.set_data(x_Wf_ns, y_Wf_ns)
#     point_Wb_2d_s.set_data(x_Wb_ns, y_Wb_ns)
#     point_COM_2d_s.set_data(x_COM_ns, y_COM_ns)

# # Create the 2D animation
# ani_2d = FuncAnimation(fig_2d, animate_2d, len(sol_lagrange.t))
# from IPython.display import HTML

# # Display the 2D animation as HTML
# HTML(ani_2d.to_jshtml(fps=fps))




In [ ]:
# Function to extract positions from solutions
def extract_positions_from_solution(solution, p_vals):
    """Extract the positions of the Front Wheel, Rear Wheel, and Center of Mass (COM)."""
    x_vals, y_vals = [], []
    
    # Loop over all time points and extract positions
    for i in range(solution.y.shape[1]):
        x, y, z = eval_point_coords(solution.y.T[i, :4], p_vals)
        x_vals.append([x[1], x[2], x[3]])  # Front wheel, Rear wheel, COM
        y_vals.append([y[1], y[2], y[3]])
    
    return np.array(x_vals), np.array(y_vals)

# Extracting positions for both "With Slip" and "No Slip" solutions
x_vals_with_slip, y_vals_with_slip = extract_positions_from_solution(sol_lagrange, p_vals)
x_vals_no_slip, y_vals_no_slip = extract_positions_from_solution(sol_lagrange_new, p_vals)

# Plotting the results
plt.figure(figsize=(10, 8))

# Plot paths for "With Slip" (dashed lines)
plt.plot(x_vals_with_slip[:, 0], y_vals_with_slip[:, 0], 'r--', label="Front Wheel Path (No Slip)")
plt.plot(x_vals_with_slip[:, 1], y_vals_with_slip[:, 1], 'm--', label="Rear Wheel Path (No Slip)")
plt.plot(x_vals_with_slip[:, 2], y_vals_with_slip[:, 2], 'b--', label="COM Path (No Slip)")

# Plot paths for "No Slip" (solid lines)
plt.plot(x_vals_no_slip[:, 0], y_vals_no_slip[:, 0], 'g-', label="Front Wheel Path (With Slip)")
plt.plot(x_vals_no_slip[:, 1], y_vals_no_slip[:, 1], 'c-', label="Rear Wheel Path (With Slip)")
plt.plot(x_vals_no_slip[:, 2], y_vals_no_slip[:, 2], 'y-', label="COM Path (With) Slip)")

# Add legend and labels
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.title("Comparison of Paths With and Without Slip (Front Wheel, Rear Wheel, COM)")
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()